In [25]:
import os
import json
import numpy as np
import scipy
import scipy.fft
import matplotlib.pyplot as plt

from scipy import signal
from scipy import integrate
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter, filtfilt, savgol_filter
from scipy.interpolate import interp1d

In [35]:
def fft(data, fs):

    N = len(data)
    T = 1.0 / fs
    
    x = np.linspace(0.0, N*T, N)
    y = scipy.fft.fft(data)

    x_f = np.linspace(0.0, 1.0/(2.0*T), N // 2)
    y_f = 2.0 / N * np.abs(y[:N // 2])

    return x, y, x_f, y_f

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def low_pass_filter(x, cutoff, fs, order,type='float'):
    if type == 'str':
        x = [float(dx) for dx in x]
    nyquist = fs / 2
    b, a = butter(order, cutoff / nyquist)

    filtered = filtfilt(b, a, x, method='gust')
    if type == 'str':
        return [str(s) for s in filtered]
    else:
        return filtered

def NormaliseCrossCorrelation(a,v):
    M = len(a)
    N = len(v)

    if type(a) is list: 
        a = np.array(a)
    if type(v) is list:
        v = np.array(v)
    
    v_mu = np.mean(v)

    ncc_series = []
    for i in range(M-N+1):
        _a = a[i:i+N]
        a_sum = _a - np.mean(_a)
        v_sum = v - v_mu
        nr = 1/N * np.sum(a_sum*v_sum) / np.sqrt(np.var(_a)*np.var(v))
        ncc_series.append(nr)

    return ncc_series

def fix(data, max_len):
    data_len = len(data) - 1
    max_len = max_len - 1
    dev = max_len - data_len
    x = np.linspace(0, data_len, num = data_len + 1, endpoint=True)
    y = np.array(data)
    f = interp1d(x, y, kind = 'slinear')
    xnew = np.linspace(0, data_len, num = data_len + dev + 1, endpoint=True)
    return f(xnew)

def Variability(a):
    result = []
    for j in range(len(a[1])):
        col_array = [a[i][j] for i in range(len(a))]
        result.append((max(col_array) - min(col_array)) / np.mean(col_array))
    return result  

def SampEn(L, m, r):
    N = len(L)
    B = 0.0
    A = 0.0

    # Split time series and save all templates of length m
    xmi = np.array([L[i : i + m] for i in range(N - m)])
    xmj = np.array([L[i : i + m] for i in range(N - m + 1)])

    # Save all matches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])

    # Similar for computing A
    m += 1
    xm = np.array([L[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])

    # Return SampEn
    return -np.log(A / B)

def CheckUpsideDown(signal):

    signalU = -1 * signal

    peaks, _ = find_peaks(signal[0:1000], height = 75, distance = 25)
    peaksU, _ = find_peaks(signalU[0:1000], height = 75, distance = 25)

    if len(peaks) > len(peaksU):
        return signalU, True
    else:
        return signal, False

def Outliers(data):
    q1 = np.quantile(data, 0.25)  # Calculate the first quartile
    q3 = np.quantile(data, 0.75)  # Calculate the third quartile
    iqr = q3 - q1  # Calculate the interquartile range
    threshold_low = q1 - 1.5 * iqr
    threshold_high = q3 + 1.5 * iqr

    outliers = np.where((data < threshold_low) | (data > threshold_high))[0]  # Calculate the index of outliers
    if outliers.size > 0:
        turnpoint = outliers[0] # Calculate the position of the first outlier
        isTurn = True
    else:
        turnpoint = False
        isTurn = False

    return isTurn, turnpoint, outliers

def FindMid(leftGyro, rightGyro, leftPeaks, rightPeaks):
    
    leftCutOffL = leftPeaks[1] # Find the target signal start, is middle of all signals
    # leftCutOffR = leftPeaks[np.where(leftCutOffL <= leftPeaks)[0][5]]
    leftCutOffR = leftCutOffL + 200
    rightCutOffL = rightPeaks[np.where(leftCutOffL < rightPeaks)[0][0]]
    # rightCutOffR = rightPeaks[np.where(leftCutOffL < rightPeaks)[0][5]]
    rightCutOffR = rightCutOffL + 200

    leftArray = leftPeaks[np.where((leftCutOffL <= leftPeaks) & (leftPeaks <= rightCutOffR))]
    rightArray = rightPeaks[np.where((leftCutOffL <= rightPeaks) & (rightPeaks <= rightCutOffR))]

    # 新的轉彎檢測 2023-03-10
    isRightTurn, rightTurnPoint, _ = Outliers(rightGyro[rightArray]) # Detect if the right foot contains an outlier
    isLeftTurn, leftTurnPoint, _ = Outliers(leftGyro[leftArray]) # Detect if the left foot contains an outlier

    if isRightTurn == True:
        turnPoint = rightArray[rightTurnPoint]

    if isLeftTurn == True:
        turnPoint = leftArray[leftTurnPoint]

    if isLeftTurn == True or isRightTurn == True:    
        leftCutOffL = leftPeaks[np.where(turnPoint < leftPeaks)[0][0]]
        # leftCutOffR = leftPeaks[np.where(turnPoint < leftPeaks)[0][5]]
        leftCutOffR = leftCutOffL + 650
        rightCutOffL = rightPeaks[np.where(leftCutOffL < rightPeaks)[0][0]]
        # rightCutOffR = rightPeaks[np.where(leftCutOffL < rightPeaks)[0][5]]
        rightCutOffR = rightCutOffL + 650
        isTurn = True
    else:
        isTurn = False

    # # 舊的轉彎檢測
    # isTurn = False
    # isLeftTurn = False
    # isRightTurn = False
    # leftMean = np.mean(leftGyro[leftArray])
    # rightMean = np.mean(rightGyro[rightArray])

    # for i in rightArray:
    #     if rightMean - rightGyro[i] > 80:
    #         turnPoint = i
    #         isRightTurn = True

    # for i in leftArray:
    #     if leftMean - leftGyro[i] > 80:
    #         turnPoint = i
    #         isLeftTurn = True
    #         isTurn = True

    return isTurn, leftCutOffL, leftCutOffR, rightCutOffL, rightCutOffR

def FindValleys(data, peaks, direction):
    valleys = []

    if direction == 'right':
        for i in range(len(peaks) - 1):
            flag = 0 
            for j in range(peaks[i], peaks[i + 1]):
                slope = data[j + 1] - data[j]
                
                while slope > 0:
                    valleys.append(j)
                    flag = 1
                    break

                if flag == 1:
                    break

    if direction == 'left':
        for i in range(len(peaks) - 1):
            flag = 0 
            for j in range(peaks[i], peaks[i - 1], -1):
                slope = data[j] - data[j - 1]
                
                while slope < 0:
                    valleys.append(j)
                    flag = 1
                    break

                if flag == 1:
                    break
    return valleys

In [39]:
rootpath = './c.台南立人里/投稿版本_data/低衰/'
# rootpath = r'C:\Users\wwf\Desktop\Data\00.Python\四群資料\SVM & KNN\分類\衰弱'
all_file_name = os.listdir(rootpath)
low_swing_time = []
low_stance_time = []
for _ in all_file_name:
    name = _.split('.')[0]
    path = f'{rootpath}/{_}'
    print(path)
    with open(path) as file:
        Gait_Dict = json.load(file)
    
    # with open(Data, 'r') as f:
    # Gait_Dict = json.loads(f.read())


    # Access to the raw signal of each axis
    leftGyroZRaw, leftCheck = CheckUpsideDown(np.array(Gait_Dict['LeftFootGyro']['z']))
    leftAccXRaw = np.array(Gait_Dict['LeftFootAcc']['x'])

    rightGyroZRaw, rightCheck = CheckUpsideDown(np.array(Gait_Dict['RightFootGyro']['z']))
    rightAccXRaw = np.array(Gait_Dict['RightFootAcc']['x'])

    leftRSSRaw = np.sqrt(np.array(Gait_Dict['LeftFootAcc']['x']) ** 2 + (np.array(Gait_Dict['LeftFootAcc']['y'])) ** 2 + np.array(Gait_Dict['LeftFootAcc']['z']) ** 2)
    rightRSSRaw = np.sqrt(np.array(Gait_Dict['RightFootAcc']['x']) ** 2 + (np.array(Gait_Dict['RightFootAcc']['y'])) ** 2 + np.array(Gait_Dict['RightFootAcc']['z']) ** 2)

    # Check for signal inversion, left foot
    if leftCheck:
        leftMagRaw = -1 * (np.array(Gait_Dict['LeftFootMag']['z']))
    else:
        leftMagRaw = np.array(Gait_Dict['LeftFootMag']['z'])

    # Check for signal inversion, right foot
    if rightCheck:
        rightMagRaw = -1 * (np.array(Gait_Dict['RightFootMag']['z']))
    else:
        rightMagRaw = np.array(Gait_Dict['RightFootMag']['z'])

    # Use SG filter for gyroscope signal and Euler angle signal
    leftGyroZ = savgol_filter(leftGyroZRaw, 21, 3)
    rightGyroZ = savgol_filter(rightGyroZRaw, 21, 3)

    leftMag = savgol_filter(leftMagRaw, 21, 3)
    rightMag = savgol_filter(rightMagRaw, 21, 3)

    fs = 100

    _, _, leftFFTx, leftFFTy = fft(leftRSSRaw, fs)

    _, _, rightFFTx, rightFFTy = fft(rightRSSRaw, fs)

    # Use Butterworth filter to the acceleration signal
    # Parameters can use 20, 100, 8 or 15, 100, 4
    leftRSS = low_pass_filter(leftRSSRaw, 15, fs, 4)
    rightRSS = low_pass_filter(rightRSSRaw, 15, fs, 4)

    # Mark gait feature points
    leftToeOff, _ = rightFootFlat, _ = find_peaks(rightMag, distance = 80)
    rightToeOff, _ = leftFootFlat, _ = find_peaks(leftMag, distance = 80)

    leftInitialContact, _ = find_peaks(leftGyroZ, height = 100, distance = 80)
    rightInitialContact, _ = find_peaks(rightGyroZ, height = 100, distance = 80)

    leftFeetAdjacent = FindValleys(leftGyroZ, leftInitialContact, 'left')
    rightFeetAdjacent = FindValleys(rightGyroZ, rightInitialContact, 'left')

    leftTibiaVertical = FindValleys(leftMag, leftFootFlat, 'right')
    rightTibiaVertical = FindValleys(rightMag, rightFootFlat, 'right')

    # Confirm the signal band used
    isTurn, stopLL, stopLR, stopRL, stopRR = FindMid(leftGyroZ, rightGyroZ, leftInitialContact, rightInitialContact)
    # print(stopLL, stopLR, stopRL, stopRR)

    # Confirm if the signal band consist turn
    # print(isTurn)

    # leftRSS_ = leftRSS[stopLL:stopLR]
    # rightRSS_ = rightRSS[stopRL:stopRR]

    leftFootLeftCutOff = np.where((stopLL <= leftInitialContact) & (leftInitialContact <= stopLR))[0][0]
    leftFootRightCutOff = np.where((stopLL <= leftInitialContact) & (leftInitialContact <= stopLR))[0][-1] + 1
    leftInitialContact_ = leftInitialContact[leftFootLeftCutOff:leftFootRightCutOff] - stopLL

    rightFootLeftCutOff = np.where((stopRL <= rightInitialContact) & (rightInitialContact <= stopRR))[0][0]
    rightFootRightCutOff = np.where((stopRL <= rightInitialContact) & (rightInitialContact <= stopRR))[0][-1] + 1
    rightInitialContact_ = rightInitialContact[rightFootLeftCutOff:rightFootRightCutOff] - stopRL

    leftToeOff_ = leftToeOff[np.where(leftToeOff < stopLL)[0][-1] : np.where(leftToeOff < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL
    leftFeetAdjacent_ = leftFeetAdjacent[np.where(leftFeetAdjacent < stopLL)[0][-1] : np.where(leftFeetAdjacent < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL
    leftTibiaVertical_ = leftTibiaVertical[np.where(leftTibiaVertical < stopLL)[0][-1] : np.where(leftTibiaVertical < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL
    leftFootFlat_ = leftFootFlat[np.where(leftFootFlat < stopLL)[0][-1] : np.where(leftFootFlat < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL

    rightToeOff_ = rightToeOff[np.where(rightToeOff < stopRL)[0][-1] : np.where(rightToeOff < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL
    rightFeetAdjacent_ = rightFeetAdjacent[np.where(rightFeetAdjacent < stopRL)[0][-1] : np.where(rightFeetAdjacent < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL
    rightTibiaVertical_ = rightTibiaVertical[np.where(rightTibiaVertical < stopRL)[0][-1] : np.where(rightTibiaVertical < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL
    rightFootFlat_ = rightFootFlat[np.where(rightFootFlat < stopRL)[0][-1] : np.where(rightFootFlat < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL

    # Calculate step time and swing time ratio
    leftSwingTime = np.mean((leftInitialContact_ - leftToeOff_)[1:] / np.diff(leftToeOff_))
    leftStepTime = 1 - leftSwingTime

    rightSwingTime = np.mean((rightInitialContact_ - rightToeOff_)[1:] / np.diff(rightToeOff_))
    rightStepTime = 1 - rightSwingTime

    swingInGaitCycle = (leftSwingTime + rightSwingTime) / 2
    stepInGaitCycle = (leftStepTime + rightStepTime) / 2
    low_stance_time.append(stepInGaitCycle)
    low_swing_time.append(swingInGaitCycle)
    print(f'leftSwingTime：{leftSwingTime}')
    print(f'leftSwingTime：{rightSwingTime}')

./c.台南立人里/投稿版本_data/低衰//102003.json
leftSwingTime：0.40425999121651296
leftSwingTime：0.3703807390817469
./c.台南立人里/投稿版本_data/低衰//102005.json
leftSwingTime：0.37933052434456926
leftSwingTime：0.391304347826087
./c.台南立人里/投稿版本_data/低衰//102007.json
leftSwingTime：0.37380693487978733
leftSwingTime：0.4
./c.台南立人里/投稿版本_data/低衰//102009.json
leftSwingTime：0.35833333333333334
leftSwingTime：0.415929203539823
./c.台南立人里/投稿版本_data/低衰//102013.json
leftSwingTime：0.4351851851851852
leftSwingTime：0.33636363636363636
./c.台南立人里/投稿版本_data/低衰//102714.json
leftSwingTime：0.3957677699403147
leftSwingTime：0.4075940860215054
./c.台南立人里/投稿版本_data/低衰//102715.json
leftSwingTime：0.34775910364145657
leftSwingTime：0.36893203883495146
./c.台南立人里/投稿版本_data/低衰//102716.json
leftSwingTime：0.375
leftSwingTime：0.3854166666666667
./c.台南立人里/投稿版本_data/低衰//102717.json
leftSwingTime：0.41174554928354323
leftSwingTime：0.37894736842105264
./c.台南立人里/投稿版本_data/低衰//102718.json
leftSwingTime：0.34855015673981193
leftSwingTime：0.3999477533960293


In [41]:
rootpath = './c.台南立人里/投稿版本_data/高衰/'
# rootpath = r'C:\Users\wwf\Desktop\Data\00.Python\四群資料\SVM & KNN\分類\衰弱'
all_file_name = os.listdir(rootpath)
hight_swing_time = []
hight_stance_time = []
for _ in all_file_name:
    name = _.split('.')[0]
    path = f'{rootpath}/{_}'
    print(path)
    with open(path) as file:
        Gait_Dict = json.load(file)
    
    # with open(Data, 'r') as f:
    # Gait_Dict = json.loads(f.read())


    # Access to the raw signal of each axis
    leftGyroZRaw, leftCheck = CheckUpsideDown(np.array(Gait_Dict['LeftFootGyro']['z']))
    leftAccXRaw = np.array(Gait_Dict['LeftFootAcc']['x'])

    rightGyroZRaw, rightCheck = CheckUpsideDown(np.array(Gait_Dict['RightFootGyro']['z']))
    rightAccXRaw = np.array(Gait_Dict['RightFootAcc']['x'])

    leftRSSRaw = np.sqrt(np.array(Gait_Dict['LeftFootAcc']['x']) ** 2 + (np.array(Gait_Dict['LeftFootAcc']['y'])) ** 2 + np.array(Gait_Dict['LeftFootAcc']['z']) ** 2)
    rightRSSRaw = np.sqrt(np.array(Gait_Dict['RightFootAcc']['x']) ** 2 + (np.array(Gait_Dict['RightFootAcc']['y'])) ** 2 + np.array(Gait_Dict['RightFootAcc']['z']) ** 2)

    # Check for signal inversion, left foot
    if leftCheck:
        leftMagRaw = -1 * (np.array(Gait_Dict['LeftFootMag']['z']))
    else:
        leftMagRaw = np.array(Gait_Dict['LeftFootMag']['z'])

    # Check for signal inversion, right foot
    if rightCheck:
        rightMagRaw = -1 * (np.array(Gait_Dict['RightFootMag']['z']))
    else:
        rightMagRaw = np.array(Gait_Dict['RightFootMag']['z'])

    # Use SG filter for gyroscope signal and Euler angle signal
    leftGyroZ = savgol_filter(leftGyroZRaw, 21, 3)
    rightGyroZ = savgol_filter(rightGyroZRaw, 21, 3)

    leftMag = savgol_filter(leftMagRaw, 21, 3)
    rightMag = savgol_filter(rightMagRaw, 21, 3)

    fs = 100

    _, _, leftFFTx, leftFFTy = fft(leftRSSRaw, fs)

    _, _, rightFFTx, rightFFTy = fft(rightRSSRaw, fs)

    # Use Butterworth filter to the acceleration signal
    # Parameters can use 20, 100, 8 or 15, 100, 4
    leftRSS = low_pass_filter(leftRSSRaw, 15, fs, 4)
    rightRSS = low_pass_filter(rightRSSRaw, 15, fs, 4)

    # Mark gait feature points
    leftToeOff, _ = rightFootFlat, _ = find_peaks(rightMag, distance = 80)
    rightToeOff, _ = leftFootFlat, _ = find_peaks(leftMag, distance = 80)

    leftInitialContact, _ = find_peaks(leftGyroZ, height = 100, distance = 80)
    rightInitialContact, _ = find_peaks(rightGyroZ, height = 100, distance = 80)

    leftFeetAdjacent = FindValleys(leftGyroZ, leftInitialContact, 'left')
    rightFeetAdjacent = FindValleys(rightGyroZ, rightInitialContact, 'left')

    leftTibiaVertical = FindValleys(leftMag, leftFootFlat, 'right')
    rightTibiaVertical = FindValleys(rightMag, rightFootFlat, 'right')

    # Confirm the signal band used
    isTurn, stopLL, stopLR, stopRL, stopRR = FindMid(leftGyroZ, rightGyroZ, leftInitialContact, rightInitialContact)
    # print(stopLL, stopLR, stopRL, stopRR)

    # Confirm if the signal band consist turn
    # print(isTurn)

    # leftRSS_ = leftRSS[stopLL:stopLR]
    # rightRSS_ = rightRSS[stopRL:stopRR]

    leftFootLeftCutOff = np.where((stopLL <= leftInitialContact) & (leftInitialContact <= stopLR))[0][0]
    leftFootRightCutOff = np.where((stopLL <= leftInitialContact) & (leftInitialContact <= stopLR))[0][-1] + 1
    leftInitialContact_ = leftInitialContact[leftFootLeftCutOff:leftFootRightCutOff] - stopLL

    rightFootLeftCutOff = np.where((stopRL <= rightInitialContact) & (rightInitialContact <= stopRR))[0][0]
    rightFootRightCutOff = np.where((stopRL <= rightInitialContact) & (rightInitialContact <= stopRR))[0][-1] + 1
    rightInitialContact_ = rightInitialContact[rightFootLeftCutOff:rightFootRightCutOff] - stopRL

    leftToeOff_ = leftToeOff[np.where(leftToeOff < stopLL)[0][-1] : np.where(leftToeOff < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL
    leftFeetAdjacent_ = leftFeetAdjacent[np.where(leftFeetAdjacent < stopLL)[0][-1] : np.where(leftFeetAdjacent < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL
    leftTibiaVertical_ = leftTibiaVertical[np.where(leftTibiaVertical < stopLL)[0][-1] : np.where(leftTibiaVertical < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL
    leftFootFlat_ = leftFootFlat[np.where(leftFootFlat < stopLL)[0][-1] : np.where(leftFootFlat < stopLL)[0][-1] + len(leftInitialContact_)] - stopLL

    rightToeOff_ = rightToeOff[np.where(rightToeOff < stopRL)[0][-1] : np.where(rightToeOff < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL
    rightFeetAdjacent_ = rightFeetAdjacent[np.where(rightFeetAdjacent < stopRL)[0][-1] : np.where(rightFeetAdjacent < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL
    rightTibiaVertical_ = rightTibiaVertical[np.where(rightTibiaVertical < stopRL)[0][-1] : np.where(rightTibiaVertical < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL
    rightFootFlat_ = rightFootFlat[np.where(rightFootFlat < stopRL)[0][-1] : np.where(rightFootFlat < stopRL)[0][-1] + len(rightInitialContact_)] - stopRL

    # Calculate step time and swing time ratio
    leftSwingTime = np.mean((leftInitialContact_ - leftToeOff_)[1:] / np.diff(leftToeOff_))
    leftStepTime = 1 - leftSwingTime

    rightSwingTime = np.mean((rightInitialContact_ - rightToeOff_)[1:] / np.diff(rightToeOff_))
    rightStepTime = 1 - rightSwingTime

    swingInGaitCycle = (leftSwingTime + rightSwingTime) / 2
    stepInGaitCycle = (leftStepTime + rightStepTime) / 2
    hight_stance_time.append(stepInGaitCycle)
    hight_swing_time.append(swingInGaitCycle)
    print(f'leftSwingTime：{leftSwingTime}')
    print(f'leftSwingTime：{rightSwingTime}')

./c.台南立人里/投稿版本_data/高衰//102001.json
leftSwingTime：0.35185185185185186
leftSwingTime：0.4074074074074074
./c.台南立人里/投稿版本_data/高衰//102002.json
leftSwingTime：0.3706896551724138
leftSwingTime：0.375
./c.台南立人里/投稿版本_data/高衰//102004.json
leftSwingTime：0.4020716685330347
leftSwingTime：0.3494449583718779
./c.台南立人里/投稿版本_data/高衰//102720.json
leftSwingTime：0.3333333333333333
leftSwingTime：0.43975903614457834
./c.台南立人里/投稿版本_data/高衰//102722.json
leftSwingTime：0.3707865168539326
leftSwingTime：0.375
./c.台南立人里/投稿版本_data/高衰//110325.json
leftSwingTime：0.3963963963963964
leftSwingTime：0.35454545454545455
./c.台南立人里/投稿版本_data/高衰//111006.json
leftSwingTime：nan
leftSwingTime：nan


C:\Users\jupiter\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\jupiter\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [46]:
import scipy.stats as stats
print(f'low_stance_time：{low_stance_time}')
print(f'low_stance_time：{low_swing_time}')
print(f'hight_stance_time：{hight_stance_time[0:-1]}')
print(f'hight_stance_time：{hight_swing_time[0:-1]}')
print(f'swing_time：{stats.ttest_ind(low_swing_time, hight_swing_time[0:-1], equal_var=False)}')
print(f'stance_time：{stats.ttest_ind(low_stance_time, hight_stance_time[0:-1], equal_var=False)}')


low_stance_time：[0.6126796348508701, 0.6146825639146719, 0.6130965325601063, 0.6128687315634218, 0.6142255892255892, 0.59831907201909, 0.641654428761796, 0.6197916666666666, 0.6046535411477021, 0.6257510449320793, 0.6186865583353369, 0.6384203591102204, 0.6182805295255095, 0.6252329116406787, 0.6552706552706553, 0.6086547452080411]
low_stance_time：[0.38732036514912993, 0.3853174360853281, 0.3869034674398937, 0.3871312684365782, 0.38577441077441077, 0.40168092798091004, 0.358345571238204, 0.38020833333333337, 0.3953464588522979, 0.3742489550679206, 0.3813134416646631, 0.36157964088977956, 0.3817194704744905, 0.3747670883593214, 0.3447293447293447, 0.39134525479195886]
hight_stance_time：[0.6203703703703703, 0.6271551724137931, 0.6242416865475436, 0.6134538152610443, 0.6271067415730337, 0.6245290745290746]
hight_stance_time：[0.37962962962962965, 0.3728448275862069, 0.3757583134524563, 0.38654618473895586, 0.3728932584269663, 0.3754709254709255]
swing_time：Ttest_indResult(statistic=0.63380